# Most ATP wins

### Create a race bar for most cumulative ATP wins since 1968. For each month the race bar will show the 10 players with the most cumulative ATP wins up until that month. the dataset is from: "https://www.kaggle.com/firasbayram/atp-matches-1968-2018-cleaned"

## Import packages

In [ ]:
import pandas as pd
import numpy as np

## Read the file

In [ ]:
col_list=['tourney_date', 'winner_id', 'winner_name']
path="C:\\Users\\Ilay\\Documents\\notebooks\\datasets\\ATP\\ATP2.csv"
atp = pd.read_csv(path, parse_dates=['tourney_date'], usecols=col_list)
atp['month_year'] = atp['tourney_date'].dt.to_period('M')
print(atp.columns)
print(atp.head())
print(atp.shape)

## Create total wins per month coulumn

In [ ]:
atp['wins']=1
atp_grouped= atp.groupby(['month_year', 'winner_id', 'winner_name'], as_index=False)['wins'].count().sort_values(['month_year', 'wins'], ascending=[True, False])
atp_grouped.columns= ['month_year', 'winner_id', 'winner_name', 'Total_month_wins']
atp_grouped.reset_index(drop=True, inplace=True)
print(atp_grouped.head())
print(atp_grouped.shape)

## Create total wins column

In [ ]:
for id, group in atp_grouped.groupby('winner_id'):
    atp_grouped.loc[atp_grouped['winner_id']==id, 'total_wins']=group['Total_month_wins'].cumsum()

atp_grouped=atp_grouped.drop(columns='Total_month_wins')     
print(atp_grouped.head())

## Choose the Top 10 for each month

In [ ]:
atp_final= pd.DataFrame(columns=atp_grouped.columns)
check= atp_grouped.head(10)

for month, group in atp_grouped.groupby('month_year'):
    to_add=check.append(group.head(10))
    to_add=to_add.groupby(['winner_id', 'winner_name'], as_index=False).max().sort_values(by=['total_wins'], ascending=False)
    to_add['month_year']=month
    to_add=to_add[atp_grouped.columns].head(10)
    atp_final=atp_final.append(to_add).reset_index(drop=True)
    check=to_add.reset_index(drop=True)
    
print(atp_final.head())
print(atp_final.shape)

## Import additional packages

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
import random

##  Attach random number to each player and create color map to the numbers

In [ ]:
random.seed(21)
atp_final=atp_final.set_index('month_year')['1982':].reset_index(drop=False) # The data was too big for the animation so it will strat running since 1982
print(atp_final.head())

names= atp_final['winner_name'].unique()
print(names[:], len(names))
rand_num=np.empty(len(names))

for i in range(len(names)):
    rand_num[i]= random.randint(1,10)
print(rand_num[:], len(rand_num))

map= dict(zip(names,rand_num))

atp_final['rand_num']=atp_final['winner_name']

atp_final=atp_final.replace({'rand_num': map}) 
print(atp_final.head())

colors = dict(zip(
    [1,2,3,4,5,6,7,8,9,10],
    ['#adb0ff', '#ffb3ff', '#90d595', '#e48381','#aafbff', '#f7bb5f', '#eafb50', 'red', 'blue', 'gray']
))
group_lk = atp_final.set_index('winner_name')['rand_num'].to_dict()

# Create the animation

In [ ]:
frames= atp_final['month_year'].unique()
print(frames)

def draw_barchart(month_year):
    selected_month = (atp_final[atp_final['month_year']==(month_year)].sort_values(by=['total_wins'], ascending=False).head(10))
    ax.clear()
    ax.barh(selected_month['winner_name'], selected_month['total_wins'], color=[colors[group_lk[x]] for x in selected_month['winner_name']])
    ax.invert_yaxis()
    dx = selected_month['total_wins'].max()
    for i, (total_wins, winner_name) in enumerate(zip(selected_month['total_wins'], selected_month['winner_name'])):
        ax.text(total_wins, i,     winner_name,            ha='right')  # Tokyo: name
        ax.text(total_wins, i,     total_wins,           ha='left')   # 38194.2: value
    # Add year right middle portion of canvas
    ax.text(1, 0.4, month_year, transform=ax.transAxes, size=30, ha='right')
    ax.text(0, 1.06, 'total_wins', transform=ax.transAxes, size=12, color='#777777')
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_ticks_position('top')
    ax.tick_params(axis='x', colors='#777777', labelsize=12)
    ax.set_yticks([])
    ax.margins(0, 0.01)
    ax.grid(which='major', axis='x', linestyle='-')
    ax.set_axisbelow(True)
    ax.text(0, 1.12, 'most Tennis wins in history',
            transform=ax.transAxes, size=24, weight=600, ha='left')
    ax.text(1, 0, 'by @pratapvardhan; credit @jburnmurdoch', transform=ax.transAxes, ha='right',
            color='#777777', bbox=dict(facecolor='white', alpha=0.8, edgecolor='white'))
    plt.box(False)


fig, ax = plt.subplots(figsize=(15, 8))
animator = animation.FuncAnimation(fig, draw_barchart, frames=frames)
HTML(animator.to_jshtml()) 